Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 5.42 ms, sys: 0 ns, total: 5.42 ms
Wall time: 12.2 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.0.243/8327/1 Dashboard: http://10.20.0.243:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 197 ms, sys: 13.9 ms, total: 211 ms
Wall time: 1.53 s


[1.2941100262976115,
 1.3606294340637946,
 1.6595713632789815,
 1.65191418028222,
 1.4273810618003635,
 1.000792805410585,
 1.1925965759058177,
 1.6470034326511267,
 1.195758682670999,
 1.624092589669064,
 1.6222638372392653,
 1.3386337716318604,
 1.0733207245638159,
 1.0493807827765838,
 1.1015308586913521,
 1.8717899734329582,
 1.753730237639386,
 1.3376763179054427,
 1.079748219246985,
 1.6253484165266152,
 1.5060408927668898,
 1.7511048551195696,
 1.8155395974029662,
 1.242006837936254,
 1.513206035573913,
 1.3585705293112287,
 1.4030200316059491,
 1.5978589014669837,
 1.7014345640992519,
 1.0451815608481723,
 1.1038039088962126,
 1.672933096513065]

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.513193042357722,
 1.3149485970085606,
 1.3638831378783158,
 1.947110027457529,
 1.5895325370141717,
 1.2666241075616265,
 1.0561630778749782,
 1.1020006836528,
 1.7227940599297566,
 1.1597653867361504,
 1.133965683597052,
 1.0580879180880662,
 1.108319374932384,
 1.6177596111354893,
 1.9633912177024673,
 1.0127475127378633,
 1.7969232662782426,
 1.323448173822989,
 1.8711482892241986,
 1.5055316166599857,
 1.691769456166925,
 1.295315417943003,
 1.704964732322671,
 1.9330276671396223,
 1.0055599765364556,
 1.0603582058331953,
 1.6453942552182852,
 1.2581881529229562,
 1.3148198030725005,
 1.8418208673891083,
 1.2920643640801126,
 1.0001837822533386,
 1.4033269521752514,
 1.021548401505446,
 1.3544414622484626,
 1.6551665123159867,
 1.9466545815770733,
 1.6737311572257454,
 1.1977834669953635,
 1.3500568189352644,
 1.8730596746605643,
 1.496362273529861,
 1.5120137126031075,
 1.1932032068112974,
 1.2731772139039434,
 1.7608616068101326,
 1.6063540619541106,
 1.6876704885861078,
 1.96

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 190 ms, sys: 27.3 ms, total: 217 ms
Wall time: 2.21 s
